# mycomposites example

In [1]:
import numpy as np

from dwave.system import DWaveSampler
from dwave.embedding import MinimizeEnergy, majority_vote, discard, weighted_random
import minorminer

from mycomposites import MinimizeEnergyComposite, FixedMinimizeEnergyComposite

## Problem setup

In [2]:
def generate_random_qubo_matrix_dict(N: int):
    Q = np.random.normal(0., 1., [N, N])
    Q = 0.5 * (Q + Q.transpose())
    Qdict = {}

    for i in range(N):
        for j in range(i, N):
            if abs(Q[i][j]) > 0.:
                Qdict[(i, j)] = Q[i][j]

    return Qdict

N = 64
Q = generate_random_qubo_matrix_dict(N)

## Using MinimizeEnergyComposite
MinimizeEnergy is an extension of EmbeddingComposite, which is a composite class that can be easily used with dwave.embedding.MinimizeEnergy that recover from chain-breaks by using greedy algorithm.

In [3]:
sampler = MinimizeEnergyComposite(DWaveSampler())

### Sampling with default arguments
```python:sample_qubo.py
MinimizeEnergyComposite.sample_qubo(self, Q, chain_strength=1.0, 
                                    chain_break_method=dwave.embedding.MinimizeEnergy,
                                    chain_break_fraction=True, embedding_parameters=None,
                                    return_embedding=None, warnings=None, **parameters)
```

In [4]:
sampleset = sampler.sample_qubo(Q, num_reads = 100)

### Lowest energy sample
By default, the record contains chain_break_fraction because the argument chain_break_fraction is True.

In [5]:
print(sampleset.first)

Sample(sample={0: 1, 1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 1, 8: 1, 9: 0, 10: 0, 11: 1, 12: 1, 13: 0, 14: 0, 15: 1, 16: 1, 17: 1, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 1, 24: 1, 25: 1, 26: 1, 27: 0, 28: 1, 29: 1, 30: 1, 31: 1, 32: 0, 33: 1, 34: 1, 35: 0, 36: 1, 37: 1, 38: 1, 39: 0, 40: 0, 41: 1, 42: 0, 43: 1, 44: 1, 45: 1, 46: 1, 47: 0, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 0, 54: 1, 55: 0, 56: 1, 57: 0, 58: 0, 59: 1, 60: 1, 61: 0, 62: 1, 63: 1}, energy=-108.01228928510142, num_occurrences=1, chain_break_fraction=0.109375)


### Sampling information
By default, sampleset.info includes 'timing' and 'problem_id', but 'embedding_context' that contains the information of graph-minor embedding automatically generated by sample_qubo.

In [6]:
print(sampleset.info)

{'timing': {'qpu_sampling_time': 31496, 'qpu_anneal_time_per_sample': 20, 'qpu_readout_time_per_sample': 274, 'qpu_access_time': 41095, 'qpu_access_overhead_time': 3594, 'qpu_programming_time': 9599, 'qpu_delay_time_per_sample': 21, 'post_processing_overhead_time': 4106, 'total_post_processing_time': 4774, 'total_real_time': 41095, 'run_time_chip': 31496, 'anneal_time_per_run': 20, 'readout_time_per_run': 274}, 'problem_id': 'a2f4d01e-4808-463b-9f34-ac0c4e182aac'}


### Sampling with return_embedding=True
```python:sample_qubo.py
MinimizeEnergyComposite.sample_qubo(self, Q, chain_strength=1.0, 
                                    chain_break_method=dwave.embedding.MinimizeEnergy,
                                    chain_break_fraction=True, embedding_parameters=None,
                                    return_embedding=True, warnings=None, **parameters)
```

In [7]:
sampleset2 = sampler.sample_qubo(Q, return_embedding=True, num_reads = 100)

### Sampling information
Now, sampleset.info includes 'timing', 'problem_id' and 'embedding_context'. The 'embedding_context' includes 'embedding', 'chain_break_method', 'embedding_parameters' and 'chain_strength'. Only 'embedding' is generated during sample_qubo, and the others are given as arguments.

In [8]:
print(sampleset2.info)

{'timing': {'qpu_sampling_time': 31496, 'qpu_anneal_time_per_sample': 20, 'qpu_readout_time_per_sample': 274, 'qpu_access_time': 41126, 'qpu_access_overhead_time': 2897, 'qpu_programming_time': 9630, 'qpu_delay_time_per_sample': 21, 'post_processing_overhead_time': 349, 'total_post_processing_time': 1041, 'total_real_time': 41126, 'run_time_chip': 31496, 'anneal_time_per_run': 20, 'readout_time_per_run': 274}, 'problem_id': '29be4d22-861a-4bff-b849-9268616be2b1', 'embedding_context': {'embedding': {0: [1216, 1344, 1350, 1342, 1334, 1088, 832, 576, 448, 320, 192, 64, 960, 1326, 1472, 1318, 1310, 1302, 709, 717, 725, 704], 1: [1264, 1136, 1213, 1261, 1008, 752, 1237, 1229, 1205, 1253, 1197, 1245, 1181, 624, 496, 1221, 1269, 368, 240, 112, 1256, 880, 1173, 1142, 1189], 2: [1203, 1075, 947, 819, 691, 563, 435, 1331, 1459, 1587, 1589, 1715, 1709, 1701, 1693, 307, 179, 1717, 51, 1697, 1825, 1597], 3: [324, 332, 340, 348, 356, 364, 372, 300, 284, 276, 268, 316, 260, 308, 380, 272, 292, 400, 5

### Sampling with the other `chain_break_method`
The argument chain_break_method can be set to one or more of the following four choices; dwave.embedding.discard, dwave.embedding.majority_vote, dwave.embedding.weighted_random, and 'dwave.embedding.MinimizeEnergy'. In the original EmbeddingComposite, it was not possible to specify MinimizeEnergy as chain_break_method.
```python:sample_qubo.py
MinimizeEnergyComposite.sample_qubo(self, Q, chain_strength=1.0, 
                                    chain_break_method=dwave.embedding.majority_vote,
                                    chain_break_fraction=True, embedding_parameters=None,
                                    return_embedding=True, warnings=None, **parameters)
```

In [9]:
sampleset3 = sampler.sample_qubo(Q, chain_break_method=majority_vote, return_embedding=True, num_reads = 100)

In [10]:
print(sampleset3.first)
print(sampleset3.info['embedding_context']['chain_break_method'])

Sample(sample={0: 1, 1: 1, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 1, 8: 1, 9: 0, 10: 0, 11: 1, 12: 1, 13: 0, 14: 0, 15: 1, 16: 1, 17: 1, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 1, 24: 1, 25: 1, 26: 1, 27: 0, 28: 1, 29: 1, 30: 1, 31: 1, 32: 0, 33: 1, 34: 1, 35: 0, 36: 1, 37: 1, 38: 1, 39: 0, 40: 0, 41: 1, 42: 0, 43: 1, 44: 1, 45: 1, 46: 1, 47: 0, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 0, 54: 1, 55: 0, 56: 1, 57: 0, 58: 0, 59: 1, 60: 1, 61: 0, 62: 1, 63: 1}, energy=-107.14922081369714, num_occurrences=1, chain_break_fraction=0.125)
majority_vote


## Using FixedMinimizeEnergyComposite
FixedMinimizeEnergy is corresponding to FixedEmbeddingComposite, which is a composite class that can be easily used with dwave.embedding.MinimizeEnergy that recover from chain-breaks by using greedy algorithm.

In [11]:
embedding = sampleset3.info['embedding_context']['embedding']
sampler2 = FixedMinimizeEnergyComposite(DWaveSampler(), embedding)

In [12]:
sampleset4 = sampler2.sample_qubo(Q, chain_break_fraction=True, num_reads=100)
print(sampleset4.first)

Sample(sample={0: 1, 1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 1, 8: 1, 9: 0, 10: 0, 11: 1, 12: 1, 13: 0, 14: 0, 15: 1, 16: 1, 17: 1, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 1, 24: 1, 25: 1, 26: 1, 27: 0, 28: 1, 29: 1, 30: 1, 31: 1, 32: 0, 33: 1, 34: 1, 35: 0, 36: 1, 37: 1, 38: 1, 39: 0, 40: 0, 41: 1, 42: 0, 43: 1, 44: 1, 45: 1, 46: 1, 47: 0, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 0, 54: 1, 55: 0, 56: 1, 57: 0, 58: 0, 59: 1, 60: 1, 61: 0, 62: 1, 63: 1}, energy=-108.01228928510142, num_occurrences=1, chain_break_fraction=0.234375)
